<a href="https://colab.research.google.com/github/octaviobedregal/maestria/blob/master/Practica04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. IMPORTAMOS LIBRERIAS**

In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import requests, zipfile, io
import csv
import math

**2. DESCARGAMOS LA BASE DE DATOS DE BANANAS Y TRANSFORMAMOS LA DATA A CSV**

In [147]:
nameDB = 'banana'

f = open ("BaseDatos-"+nameDB+'-Resumen.txt','w')
f.write('BASE DE DATOS: '+nameDB+'\n')
f.write('RESUMEN:\n')
f.write('Es un conjunto de datos artificiales donde las instancias pertenecen a varios grupos con forma de banana.\n')
f.write('Hay dos caracteristicas X1 y X2, y la etiqueta de clase Y (-1 y 1) que representa una de las dos formas de plátano en el conjunto de datos.\n')

n = 10
zipFileUrl = 'https://sci2s.ugr.es/keel/dataset/data/classification/'+nameDB+'-'+str(n)+'-fold.zip'
fileTraining = 'data/'+nameDB+'-'+str(n)+'-fold/'+nameDB+'-'+str(n)+'-tra.csv'

r = requests.get(zipFileUrl)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('data/'+nameDB+'-'+str(n)+'-fold')

nTraining = 0
nTest = 0

f.write('CONJUNTO DE DATOS - ENTRENAMIENTO:\n')
with open(fileTraining, "w") as outfile:
  csv_writer = csv.writer(outfile)
  f.write(str(['X1', 'X2', 'Y'])+'\n')
  csv_writer.writerow(['X1', 'X2', 'Y'])
  for i in range(1,(n+1)):
    fileOpen = 'data/'+nameDB+'-'+str(n)+'-fold/'+nameDB+'-'+str(n)+'-'+str(i)+'tra.dat'
    with open(fileOpen) as infile:
      lines = infile.read().splitlines()[11:]
      for line in lines:
        nTraining +=1;
        f.write(str(line.split(","))+'\n')
        csv_writer.writerow(line.split(","))

f.write('CONJUNTO DE DATOS - PRUEBAS:\n')
f.write(str(['X1', 'X2', 'Y'])+'\n')
for i in range(1,(n+1)):
  fileOpen = 'data/'+nameDB+'-'+str(n)+'-fold/'+nameDB+'-'+str(n)+'-'+str(i)+'tst.dat'
  fileTest = 'data/'+nameDB+'-'+str(n)+'-fold/'+nameDB+'-'+str(n)+'-'+str(i)+'tst.csv'
  with open(fileOpen) as infile, open(fileTest, "w") as outfile:
    lines = infile.read().splitlines()[11:]
    csv_writer = csv.writer(outfile)
    csv_writer.writerow(['X1', 'X2', 'Y'])
    for line in lines:
      nTest +=1
      f.write(str(line.split(","))+'\n')
      csv_writer.writerow(line.split(","))
  
f.close() 

**3. GENERAMOS EL MODELO**

In [148]:
capas = 10
perceptrones = 5
rate = 0.003
iteraciones = 100000

MLP = MLPClassifier(activation='relu',
                    hidden_layer_sizes = (capas, perceptrones),
                    learning_rate_init = rate,
                    max_iter = 100000)

baseDatos = pd.read_csv(fileTraining)
X = baseDatos.loc[:, baseDatos.columns != 'Y']
Y = baseDatos.Y
MLP.fit(X,Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5), learning_rate='constant',
              learning_rate_init=0.003, max_fun=15000, max_iter=100000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

**4. CALCULAMOS LAS PRECISIONES DEL ENTRENAMIENTO Y PRUEBAS**

In [149]:
f = open ('PerceptronMulticapa.txt','w')
f.write('ALUMNO: Octavio Bedregal Flores\n')
f.write('CANTIDAD DE ELEMENTOS ENTRENAMIENTO:'+str(nTraining)+'\n')
f.write('CANTIDAD DE ELEMENTOS TEST:'+str(nTest)+'\n')
f.write('NRO. CARACTERISTICAS: 2\n')
f.write('NRO. CLASES: 2\n')
f.write('PARAMETROS UTILIZADOS: 5\n')
f.write('CANTIDAD DE CAPAS OCULTAS: '+str(capas)+'\n')
predicciones_train = MLP.predict(X)
accuracy_train = accuracy_score(predicciones_train, Y)
f.write('TASA CLASIFICACIÓN ENTRENAMIENTO: '+str(round(accuracy_train*100,2))+'%\n')

sumAcuracyTest = 0
accuracys = [];

col1 =[]
col2 =[]

for i in range(1,(n+1)):
  fileTest = 'data/'+nameDB+'-'+str(n)+'-fold/'+nameDB+'-'+str(n)+'-'+str(i)+'tst.csv'
  baseDatosTest = pd.read_csv(fileTest)
  X_test = baseDatosTest.loc[:, baseDatos.columns != 'Y']
  Y_test = baseDatosTest.Y
  predicciones_test = MLP.predict(X_test)
  accuracy_test = accuracy_score(predicciones_test, Y_test)
  sumAcuracyTest += accuracy_test
  accuracys.append(accuracy_test)
  col1.append(str(i-1))
  col2.append(str(round(accuracy_test*100,2))+'%')
  f.write('FOLD '+str(i-1)+', TASA CLASIFICACIÓN PRUEBA: '+str(round(accuracy_test*100,2))+'%\n')

media = sumAcuracyTest/n
sum1 = 0
for a in accuracys:
  sum1 += (a - media)**2

desvEstandar = math.sqrt((1/(n-1))*(sum1))

col1.append('MEDIA')
col1.append('DESVIACIÓN ESTANDAR')
col2.append(str(round(media*100,2))+'%')
col2.append(str(desvEstandar))

f.write('MEDIA: '+str(round(media*100,2))+'%\n')
f.write('DESVIACIÓN ESTANDAR: '+str(desvEstandar)+'\n')
f.close() 

df = pd.DataFrame(list(zip(col1, col2)),
               columns =['FOLD', 'VALOR'])
df


,FOLD,VALOR
0,0,89.73%
1,1,90.11%
2,2,89.35%
3,3,89.35%
4,4,91.63%
5,5,89.73%
6,6,90.3%
7,7,90.68%
8,8,92.78%
9,9,92.02%
